<a href="https://colab.research.google.com/github/manohar2000/Technocollab_DataScience/blob/master/emotion_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.chdir("/content/drive/My Drive/Projects/Face_recog")
os.listdir('.')

['haarcascade_frontalface_default.xml',
 'fer2013.csv',
 '.ipynb_checkpoints',
 'fer.json',
 'fer.h5']

In [ ]:
import sys, os
import pandas as pd
import numpy as np

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [ ]:
df=pd.read_csv('fer2013.csv')

In [ ]:
# print(df.info())
# print(df["Usage"].value_counts())

# print(df.head())
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
# print(f"shape:{X_train.shape}")
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


Epoch 1/30
449/449 [==============================] - 18s 40ms/step - loss: 1.7417 - accuracy: 0.2841 - val_loss: 1.6111 - val_accuracy: 0.3530
Epoch 2/30
449/449 [==============================] - 17s 39ms/step - loss: 1.5368 - accuracy: 0.3970 - val_loss: 1.3935 - val_accuracy: 0.4589
Epoch 3/30
449/449 [==============================] - 17s 38ms/step - loss: 1.4230 - accuracy: 0.4496 - val_loss: 1.3507 - val_accuracy: 0.4876
Epoch 4/30
449/449 [==============================] - 17s 39ms/step - loss: 1.3655 - accuracy: 0.4744 - val_loss: 1.3156 - val_accuracy: 0.4937
Epoch 5/30
449/449 [==============================] - 17s 38ms/step - loss: 1.3129 - accuracy: 0.4943 - val_loss: 1.2599 - val_accuracy: 0.5238
Epoch 6/30
449/449 [==============================] - 17s 38ms/step - loss: 1.2717 - accuracy: 0.5110 - val_loss: 1.2289 - val_accuracy: 0.5294
Epoch 7/30
449/449 [==============================] - 17s 39ms/step - loss: 1.2425 - accuracy: 0.5233 - val_loss: 1.2439 - val_accuracy:

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")